In [1]:
#@title Install required libraries
!pip -q install transformers accelerate sentence-transformers --upgrade
!pip -q install python-docx pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.0 MB/s eta 0:00:00


In [2]:
#@title Adapters: HF LLM & HF Embedding
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch
from typing import List

class HFLLMAdapter:
    """
    text_chunker의 LLMClient(complete)와 호환.
    - 기본은 결정론적(temperature 없음, do_sample=False)으로 JSON 안정화.
    """
    def __init__(self, model_name="Qwen/Qwen2.5-0.5B-Instruct", max_new_tokens=700):
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            trust_remote_code=True,
        )
        self.max_new_tokens = max_new_tokens

    def complete(self, prompt: str, **gen_kwargs) -> str:
        inputs = self.tok(prompt, return_tensors="pt").to(self.model.device)
        out = self.model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=gen_kwargs.get("max_tokens", self.max_new_tokens)
        )
        text = self.tok.decode(out[0], skip_special_tokens=True)
        # 프롬프트가 앞부분에 포함되어 있으면 제거 시도
        if text.startswith(prompt):
            text = text[len(prompt):].lstrip()
        return text


class HFSentenceEmbedAdapter:
    """
    embedding 모듈의 EmbeddingClient(embed)와 호환.
    - 기본 모델은 소형/빠른 ST 모델.
    """
    def __init__(self, model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed(self, texts: List[str], **kwargs) -> List[List[float]]:
        embs = self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        return [e.tolist() for e in embs]


In [3]:
#@title Runtime config
LLM_MODEL_NAME      = "Qwen/Qwen2.5-0.5B-Instruct" #@param ["Qwen/Qwen2.5-0.5B-Instruct", "Qwen/Qwen2.5-1.5B-Instruct", "microsoft/Phi-3-mini-4k-instruct", "google/gemma-2-2b-it"] {allow-input: true}
EMBED_MODEL_NAME    = "sentence-transformers/paraphrase-MiniLM-L6-v2" #@param ["sentence-transformers/paraphrase-MiniLM-L6-v2", "intfloat/multilingual-e5-base", "BAAI/bge-m3"] {allow-input: true}
MAX_NEW_TOKENS_LLM  = 700 #@param {type:"integer"}

print("LLM :", LLM_MODEL_NAME)
print("EMB :", EMBED_MODEL_NAME)


LLM : Qwen/Qwen2.5-0.5B-Instruct
EMB : sentence-transformers/paraphrase-MiniLM-L6-v2


In [4]:
#@title Upload documents to ingest (txt, docx, pdf, hwpx 등)
from google.colab import files
from pathlib import Path

print("인제스트할 문서 파일들을 선택하세요.")
uploaded_docs = files.upload()  # 여러 개 선택 가능
doc_paths = []
for name, data in uploaded_docs.items():
    p = Path(name)
    with open(p, "wb") as f:
        f.write(data)
    doc_paths.append(str(p.resolve()))

print("✅ 업로드 완료:", doc_paths)


인제스트할 문서 파일들을 선택하세요.


Saving sample.txt to sample.txt
✅ 업로드 완료: ['/content/sample.txt']


In [5]:
#@title Run full pipeline on uploaded documents
import importlib, json, time
from pathlib import Path

file_reader   = importlib.import_module("file_reader")
text_chunker  = importlib.import_module("text_chunker")
embedding_mod = importlib.import_module("embedding")

# 모델 초기화
llm = HFLLMAdapter(LLM_MODEL_NAME, max_new_tokens=MAX_NEW_TOKENS_LLM)
embedder = HFSentenceEmbedAdapter(EMBED_MODEL_NAME)

all_records = []
t0 = time.time()

for path in doc_paths:
    print(f"\n=== Processing: {path} ===")
    # 1) 파일 로드
    try:
        text = file_reader.read_file(path)
    except Exception as e:
        print(f"[ERR] read_file failed: {e}")
        continue

    if not isinstance(text, str) or not text.strip():
        print("[WARN] Empty text; skipping.")
        continue
    print(f"[OK] Loaded text: {len(text)} chars")

    # 2) LLM 기반 청크 분할
    try:
        chunk_result = text_chunker.llm_guided_sentence_chunk(
            text,
            llm=llm,
            max_sentences_per_chunk=20,
            system_hint="논리/주제 전환 기준으로 5~20문장씩 묶고, 오직 JSON만 출력하세요."
        )
    except Exception as e:
        print(f"[ERR] chunking failed: {e}")
        continue

    if not hasattr(chunk_result, "chunks") or len(chunk_result.chunks) == 0:
        print("[WARN] No chunks; skipping.")
        continue

    print(f"[OK] Chunks: {len(chunk_result.chunks)}")

    # 3) 임베딩
    chunks = [c.text for c in chunk_result.chunks]
    offsets = [{
        "chunk_index": c.chunk_index,
        "char_start": c.char_start,
        "char_end": c.char_end,
        "word_start": c.word_start,
        "word_end": c.word_end
    } for c in chunk_result.chunks]

    try:
        records = embedding_mod.build_embeddings(
            chunks=chunks,
            file_path=path,
            filetype=Path(path).suffix.lstrip(".").lower() or "unknown",
            offsets=offsets,
            embedder=embedder
        )
    except Exception as e:
        print(f"[ERR] embedding failed: {e}")
        continue

    print(f"[OK] Embeddings: {len(records)} vectors, dim={len(records[0].vector) if records else 0}")
    all_records.extend(records)

t1 = time.time()
print(f"\n=== DONE: {len(all_records)} records from {len(doc_paths)} files in {t1-t0:.2f}s ===")

# 4) 저장 (JSONL)
out_path = Path("embeddings.jsonl")
with open(out_path, "w", encoding="utf-8") as f:
    for r in all_records:
        f.write(json.dumps({
            "vector": r.vector,
            "text": r.text,
            "meta": {
                "file_path": r.meta.file_path,
                "filetype": r.meta.filetype,
                "chunk_index": r.meta.chunk_index,
                "char_start": r.meta.char_start,
                "char_end": r.meta.char_end,
                "word_start": r.meta.word_start,
                "word_end": r.meta.word_end
            }
        }, ensure_ascii=False) + "\n")

print(f"📝 Saved embeddings to: {out_path.resolve()}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Processing: /content/sample.txt ===
[OK] Loaded text: 2167 chars
=== RAW LLM OUTPUT (first try) ===
[51] 인공지능은 우리의 미래를 결정할 수 있습니다.
[52] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[53] 인공지능은 우리의 미래를 결정할 수 있습니다.
[54] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[55] 인공지능은 우리의 미래를 결정할 수 있습니다.
[56] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[57] 인공지능은 우리의 미래를 결정할 수 있습니다.
[58] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[59] 인공지능은 우리의 미래를 결정할 수 있습니다.
[60] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[61] 인공지능은 우리의 미래를 결정할 수 있습니다.
[62] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[63] 인공지능은 우리의 미래를 결정할 수 있습니다.
[64] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[65] 인공지능은 우리의 미래를 결정할 수 있습니다.
[66] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[67] 인공지능은 우리의 미래를 결정할 수 있습니다.
[68] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[69] 인공지능은 우리의 미래를 결정할 수 있습니다.
[70] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[71] 인공지능은 우리의 미래를 결정할 수 있습니다.
[72] 인공지능은 우리에게 미래를 선언하고, 미래를 준비하는 데 필요한 도구가 됩니다.
[73]

In [6]:
#@title (Optional) Preview a few records
import json
from pathlib import Path
from itertools import islice

path = Path("embeddings.jsonl")
if not path.exists():
    print("embeddings.jsonl 이 없습니다. 이전 셀을 먼저 실행하세요.")
else:
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(islice(f, 5)):
            row = json.loads(line)
            preview = row["text"][:120].replace("\n", " ")
            print(f"[{i}] {row['meta']['filetype']} #{row['meta']['chunk_index']} | {preview}")


[0] txt #0 | 인공지능은 이제 더 이상 연구실 안의 기술이 아닙니다. 우리는 매일같이 인공지능이 적용된 제품과 서비스를 사용하고 있습니다. 스마트폰의 얼굴 인식 기능, 온라인 쇼핑의 맞춤형 추천, 내비게이션의 교통 예측까지 모두 
[1] txt #1 | 예를 들어, 단순 반복 업무를 수행하던 직군은 인공지능 시스템에 의해 대체될 가능성이 큽니다. 기업 입장에서는 효율성이 높아지지만, 노동자 입장에서는 불안정성이 증가합니다. 이러한 변화는 산업 구조와 노동 시장에 큰
[2] txt #2 | 또한 범죄 예측 시스템이나 신용 평가 시스템에서도 편향이 발견되었습니다. 이러한 문제는 기술 그 자체의 문제가 아니라, 사회적 맥락이 반영된 결과입니다. 따라서 인공지능의 설계 단계에서부터 편향을 줄이는 노력이 필요
[3] txt #3 | 자율 무기 시스템, 사이버 전쟁 기술 등이 그 예입니다. 이러한 상황은 새로운 국제 규범과 협약을 요구합니다. 그렇지 않으면 인공지능은 갈등을 심화시키는 요인이 될 수 있습니다. 동시에 인공지능은 국제 협력의 기회도
[4] txt #4 | 기술은 목적이 아니라 수단이어야 합니다. 인공지능이 인류의 미래에 긍정적으로 기여하려면, 우리가 기술의 방향성을 주도적으로 결정해야 합니다. 이러한 논의는 단순한 기술적 논의가 아니라 사회적 합의와 가치의 문제입니다
